In [36]:
import pandas as pd

In [37]:
import pickle

In [38]:
import os

In [39]:
from sklearn.metrics import accuracy_score


In [40]:
data_path = os.path.join('..', 'datacollection', 'Placement_Data_Full_Class.csv')
data = pd.read_csv(data_path)

In [41]:
data.shape

(215, 15)

In [42]:
data.isnull().sum()

sl_no              0
gender             0
ssc_p              0
ssc_b              0
hsc_p              0
hsc_b              0
hsc_s              0
degree_p           0
degree_t           0
workex             0
etest_p            0
specialisation     0
mba_p              0
status             0
salary            67
dtype: int64

In [43]:
data = data.drop(['sl_no', 'salary'],axis=1)

I am dropping salary because it has the most number of null values. Moreover, I do not understand the need of it as we are predicting the wheather they gonna placed or not and if they have the some salary then they are alrady placed and it has max num of missing values.

I am dropping the sl.no because it will not help me with anything as it just a index values. 

In [44]:
def sniff_modified(df):
    with pd.option_context("display.max_colwidth", 40):
        info = pd.DataFrame()
        info['data type'] = df.dtypes
        info['percent missing'] = df.isnull().sum()*100/len(df)
        info['No. unique'] = df.apply(lambda x: len(x.unique()))
        info['unique values'] = df.apply(lambda x: x.unique())
        return info.sort_values('data type')

In [45]:
sniff_modified(data)

,data type,percent missing,No. unique,unique values
ssc_p,float64,0.0,103,"[67.0, 79.33, 65.0, 56.0, 85.8, 55.0, 46.0, 82..."
hsc_p,float64,0.0,97,"[91.0, 78.33, 68.0, 52.0, 73.6, 49.8, 49.2, 64..."
degree_p,float64,0.0,89,"[58.0, 77.48, 64.0, 52.0, 73.3, 67.25, 79.0, 6..."
etest_p,float64,0.0,100,"[55.0, 86.5, 75.0, 66.0, 96.8, 74.28, 67.0, 91..."
mba_p,float64,0.0,205,"[58.8, 66.28, 57.8, 59.43, 55.5, 51.58, 53.29,..."
gender,object,0.0,2,"[M, F]"
ssc_b,object,0.0,2,"[Others, Central]"
hsc_b,object,0.0,2,"[Others, Central]"
hsc_s,object,0.0,3,"[Commerce, Science, Arts]"
degree_t,object,0.0,3,"[Sci&Tech, Comm&Mgmt, Others]"


In [46]:
missingData = data.isnull().sum()
if missingData.sum() == 0:
    print("No missing values.")
else:
    print("Missing values found. Need to handle them.")

No missing values.


In [47]:
def convert_to_lower(data):
    return data.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)

data = convert_to_lower(data)

now we wont be having problem with casesenisity. It will consider others and Others as same earlier it wont.


In [48]:
data['status'].unique()

array(['placed', 'not placed'], dtype=object)

this two would be our class from which we would prediacting

**Connection to datbase**

In [49]:
import sqlite3
conn = sqlite3.connect('campuss.db')
cur = conn.cursor()
data.to_sql('studentss', conn, if_exists='replace', index=False)
sqlite_data = pd.read_sql_query("SELECT * FROM studentss", conn)

sqlite_data
print(sqlite_data.head(5))

  gender  ssc_p    ssc_b  hsc_p    hsc_b     hsc_s  degree_p   degree_t  \
0      m  67.00   others  91.00   others  commerce     58.00   sci&tech   
1      m  79.33  central  78.33   others   science     77.48   sci&tech   
2      m  65.00  central  68.00  central      arts     64.00  comm&mgmt   
3      m  56.00  central  52.00  central   science     52.00   sci&tech   
4      m  85.80  central  73.60  central  commerce     73.30  comm&mgmt   

  workex  etest_p specialisation  mba_p      status  
0     no     55.0         mkt&hr  58.80      placed  
1    yes     86.5        mkt&fin  66.28      placed  
2     no     75.0        mkt&fin  57.80      placed  
3     no     66.0         mkt&hr  59.43  not placed  
4     no     96.8        mkt&fin  55.50      placed  


In [50]:
sqlite_data.columns

Index(['gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p',
       'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'status'],
      dtype='object')

In [51]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()


categoricalCols = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']
for col in categoricalCols:
    sqlite_data[col] = label_encoder.fit_transform(sqlite_data[col])

In [52]:
X= sqlite_data.drop('status',axis=1)
Y =sqlite_data['status']

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [55]:
from sklearn.linear_model import LogisticRegression

In [56]:
lr =LogisticRegression()
lr.fit(X_train,Y_train)

c:\Users\rahul\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

**how my model works on the testing data**

In [57]:
new_data = pd.DataFrame({
    'gender':0,
    'ssc_p':67.0,
    'ssc_b': 0,
    'hsc_p':91.0,
    'hsc_b':0,
    'hsc_s':1,
    'degree_p':58.0,
    'degree_t':2,
    'workex':0,
    'etest_p':55.0,
    'specialisation':1,
    'mba_p':58.8,
},index=[0])

In [58]:
probability=lr.predict(new_data)
prob = lr.predict_proba(new_data)

print(f"{prob[0][1]:2f}")


0.940190


In [59]:
conn.commit()
conn.close()

In [60]:
import pickle

In [61]:
pickle.dump(lr, open('llr.pkl', 'wb'))

In [62]:
pickle.dump(lr, open('llr.pkl', 'wb'))

loaded_model = pickle.load(open('llr.pkl', 'rb'))

y_pred_train = loaded_model.predict(X_train)

acc_score = accuracy_score(Y_train, y_pred_train)

print("acc score for the saved model:", acc_score)


acc score for the saved model: 0.8895348837209303


this accuracy score is pretty good so that why we have decided to move further with deployment

We are using here accuracy score as it my classification model.

**HOW i have done the data handling and improvemnets**:-
 - I have handle the missing values, I have drop the useless columns, converting the strings to lowercase, encoding categorical variables.

**HOW I have done choose linear regression model?**
 - lr is best for the binary search. Here, my outcome was wheather student got placed or not so thats why i choose this model.


    